In [1]:
from compute import Evaluable, anp_math, evaluable_with_unit, Var, get_unit, ureg, coupled_run, buildidpvars, eqvar, unit_conversion_factors
from inputresolver import reassigneq, eqvars, eqsonly, default_out, mdf_order, getdofs, idx_eqlist
from inputresolver import getallvars, invert_edges, resolve, reassign
from representations import bipartite_repr
from compute_utils import get_outputs, check_eqs, print_out
from pprint import pprint
import networkx as nx
import sympy as sp
import numpy as np
import openmdao.api as om

Unable to import mpi4py. Parallel processing unavailable.
Unable to import petsc4py. Parallel processing unavailable.
Unable to import petsc4py. Parallel processing unavailable.


In [67]:
def possible_outputs(expr):
    expr_expanded = sp.expand(expr)
    drop_args = set()
    solve_args = ()
    for arg in sp.preorder_traversal(expr_expanded):
        if arg.func == sp.Pow:
            base, power = arg.args
            if base.func in (sp.Symbol,Var) and power != -1:
                drop_args.add(base)
        elif arg.func in (sp.Symbol,Var) and arg not in drop_args:
            solve_args += (arg,)
    return solve_args

In [2]:
ureg.define('USD = [currency] ')

In [3]:
IFOV = Var('IFOV', 7.272e-5, 'rad')
r = Var('r', 9.257e19, 'm')
l = Var('lambda', 5e-7, 'm')
d = Var('d', 3e-5, 'm')
Q = Var('Q', 1.1)
rho = Var('rho', 1.22, never_output=True)
k = Var('k', 1.381e-23, 'J/K', never_output=True)
T = Var('T', 5785, 'K')
h = Var('h', 6.626e-34, 'J*s')
c = Var('c', 2.998e8, 'm/s', never_output=True)
tau = Var('tau', 1)
dl = Var('Delta_lambda', 2e-6, 'm')
R = Var('R', 6.96e8, 'm')
dV = Var('Delta_V', 0, 'm/s')
g = Var('g', 9.8, 'm/s^2', never_output=True)
isp = Var('I_sp', 450, 's')
ct = Var('c_t', 1163, 'USD/kg')
ms = Var('m_s', 1175, 'kg/m^2')
QE = Var('Q_E', 0.5)
Nr = Var('N_r', 25)
tau0 = Var('tau_0', 0.75)
Ti = Var('T_i', 30, 's')
a = Var('alpha', 50e3, 'USD/m^3')
SNR_req = Var('SNR_req', 10, never_output=True)

In [4]:
Y, eq1 = eqvar('Y', IFOV*r, 'm')
f, eq2 = eqvar('f', r*d/Y, 'm')
D, eq3 = eqvar('D', 2*rho*l*Q*f/d, 'm')
tr, eq4 = eqvar('theta_r', rho*l/D, 'rad')
mt, eq5 = eqvar('mt', ms*D**2, 'kg')
mi, eq6 = eqvar('m_i', mt*sp.exp(dV/(g*isp)), 'kg')
CD, eq7 = eqvar('C_D', a*D**3, 'USD')
CT, eq8 = eqvar('C_T', CD+ct*mi, 'USD')
Hl, eq9 = eqvar('H_lambda', 2*sp.pi*h*c**2/l**5*1/(sp.exp(c*h/(k*T*l))-1), 'W/m^3')
L, eq10 = eqvar('L', 1/4*R**2*Hl*tau*dl, 'W/sr')
Pin, eq11 = eqvar('P_in', sp.pi*(D/(2*r))**2*L, 'W')
Hi, eq12 = eqvar('H_i', Pin*tau0*Ti, 'J')
Np, eq13 = eqvar('N_p', Hi*l/(h*c))
SNR, eq14 = eqvar('SNR', Np*QE/sp.sqrt(Nr**2+Np*QE))
eq15 = (SNR, SNR_req)

In [71]:
eq_list=[eq1, eq2, eq3, eq4, eq5, eq6, eq7, eq8, eq9, eq10, eq11, eq12, eq13, eq14, eq15]
eqs = idx_eqlist(eq_list)
ins = getdofs(eqs)
eqv = eqvars(eqs)
dout = default_out(eqs)
order = mdf_order(eqv, dout)

In [72]:
eqvcst = {key: {elt for elt in outsetposs if not elt.never_output}
        for key, outsetposs in eqv.items()}

In [75]:
not_output = {
    8: {h,l},
    13: {Np, QE}
}
# for key, (left, right) in eqs.items():
#     not_output[key] = set()
#     for elt in left.free_symbols.union(right.free_symbols):
#         try:
#             reassigneq(left, right, elt)
#         except:
#             not_output[key].add(elt)

In [77]:
eqvcst = {key: val - not_output.get(key, set()) for key,val in eqvcst.items()}
eqvcst

{0: {IFOV, Y, r},
 1: {Y, d, f, r},
 2: {D, Q, d, f, lambda},
 3: {D, lambda, theta_r},
 4: {D, m_s, mt},
 5: {Delta_V, I_sp, m_i, mt},
 6: {C_D, D, alpha},
 7: {C_D, C_T, c_t, m_i},
 8: {H_lambda, T},
 9: {Delta_lambda, H_lambda, L, R, tau},
 10: {D, L, P_in, r},
 11: {H_i, P_in, T_i, tau_0},
 12: {H_i, N_p, h, lambda},
 13: {N_r, SNR},
 14: {SNR}}

In [78]:
G, edges_original = bipartite_repr(eqvcst)
vrs = getallvars(eqs)
eqns = eqs.keys()
edges = invert_edges(edges_original)

In [105]:
sol = resolve(eqns, vrs, edges, 3, not_input=[l])

C [{lambda, 2, 3, D}]
1 0.0 [2.0]
C []
2 0.0 []
C []
3 0.0 []


In [104]:
sol = nx.bipartite.maximum_matching(G, top_nodes=eqvcst.keys())

In [106]:
new_eqs=reassign(eqs, dict(sol))
new_ins = getdofs(new_eqs)
new_order = mdf_order(eqv, dict(sol))

In [107]:
new_order

[14, 13, 11, 12, 8, 7, 6, 4, 5, 3, 10, 9, 1, 2, 0]

In [113]:
new_eqs

{0: (IFOV, Y/r),
 1: (d, Y*f/r),
 2: (Q, D*d/(2*f*lambda*rho)),
 3: (theta_r, lambda*rho/D),
 4: (mt, D**2*m_s),
 5: (Delta_V, I_sp*g*log(m_i/mt)),
 6: (alpha, C_D/D**3),
 7: (C_T, C_D + c_t*m_i),
 8: (H_lambda, 2*pi*c**2*h/(lambda**5*(exp(c*h/(T*k*lambda)) - 1))),
 9: (tau, 4.0*L/(Delta_lambda*H_lambda*R**2)),
 10: (L, 4*P_in*r**2/(pi*D**2)),
 11: (T_i, H_i/(P_in*tau_0)),
 12: (lambda, N_p*c*h/H_i),
 13: (N_r, -sqrt(N_p*Q_E*(N_p*Q_E - SNR**2)/SNR**2)),
 14: (SNR, SNR_req)}

In [109]:
prob = om.Problem()
model = prob.model
counter = coupled_run(new_eqs, new_order, (), model, model, 0)

In [110]:
buildidpvars(new_ins, model)

In [111]:
prob.setup()
prob.run_model()

C:\Users\johan\Miniconda2\envs\mdo\lib\site-packages\autograd\tracer.py:48: RuntimeWarning: overflow encountered in exp
  return f_raw(*args, **kwargs)
<lambdifygenerated-123>:2: RuntimeWarning: divide by zero encountered in double_scalars
  return (4.0*L/(Delta_lambda*H_lambda*R**2))


In [112]:
out = get_outputs(new_eqs, model)
print_out(out)

{R: '6.96e+08',
 k: '1.381e-23',
 N_p: '0.87',
 d: '1.171e-21',
 H_lambda: '0',
 D: '0.21',
 g: '9.80',
 mt: '50.21',
 lambda: '3.336e-25',
 Y: '0.49',
 c_t: '1163',
 T: '5785',
 h: '6.626e-34',
 rho: '1.22',
 L: '2.345e+41',
 alpha: '86.70',
 I_sp: '450',
 H_i: '0.52',
 C_T: '712.22',
 Q: '1340',
 r: '9.257e+19',
 f: '0.22',
 m_s: '1175',
 tau: 'inf',
 Delta_V: '-1.944e+04',
 P_in: '0.92',
 SNR_req: '10',
 N_r: 'nan',
 m_i: '0.61',
 SNR: '10',
 c: '2.998e+08',
 Delta_lambda: '2e-06',
 Q_E: '0.50',
 tau_0: '0.75',
 C_D: '0.77',
 T_i: '0.75',
 theta_r: '1.969e-24',
 IFOV: '5.276e-21'}

In [89]:
outs = {str(key): val for key,val in out.items()}

In [90]:
out[l]

2.1422598507886146e-24